### Dynamic Programming with shortage cost 

#### How to run each test instance


**Instance 1:**  

    1) Use T_1 = 10, high shortage costs (st_1) for 10 instances, average shortage costs (st_2) for 10 instances and low    shortage costs (st_3) for 10 instances    
    
    2) Click 'Kernel' in the menu and select 'Restart & Run All' to run the program once.
    
    3) Run the model 30 times to obtain 30 instances

**Instance 2:** 
    
     1) Use T_2 = 20,  high shortage costs (st_1) for 10 instances, average shortage costs (st_2) for 10 instances and low    shortage costs (st_3) for 10 instances    
    
     2) Click 'Kernel' in the menu and select 'Restart & Run All' to run the program once.
     
     3) Run the model 30 times to obtain 30 instances

**Instance 3:** 
    
     1) Use T_1 = 30,  high shortage costs (st_1) for 10 instances, average shortage costs (st_2) for 10 instances and low    shortage costs (st_3) for 10 instances    
    
     2) Click 'Kernel' in the menu and select 'Restart & Run All' to run the program once.
     
     3) Run the model 30 times to obtain 30 instances

#### Import Modules

In [120]:
# reading file
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

#math and random operations
import random as rd
import math
import numpy as np

# Generating graphs
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt

#### Define DP Stages and States and variables

In [121]:
rd.seed(2844) #Used for random generator seed

T_1,T_2,T_3 = 10, 20, 30 #T values for the final planning horizon period

S = 40 #storage capacity
Q = 40 # max number of units ordered per week
I0 = 0 #starting inventory of 0

# costs
def generateVariables(T):
    ct = [round(rd.uniform(10,20),1) for i in range(T)] # unit ordering costs
    ht = [round(rd.uniform(5,15),1) for i in range(T)] # holding ordering costs
    dt = [int(rd.uniform(0,20)) for i in range(T)] # demand for each period
    st_1 = [int(rd.uniform(0,20)) for i in range(T)] # demand for each period 
    st_2 = [int(rd.uniform(0,20)) for i in range(T)] # demand for each period 
    st_3 = [int(rd.uniform(0,20)) for i in range(T)] # demand for each period 
    return [ct, ht, dt, st_1, st_2, st_3]

#### Helper functions

In [122]:
def dummy_fill_In(table, tablenum):
    if (tablenum == 1):
        table.append([i for i in range(0,Q+1)])
    else:    
        for row in range(0,S+1):
            list = []
            for col in range(0,Q+1):
                list.append(col)
            table.append(list)
        
#test instance 1 use st_1, test instance 2 use st_2, test instance 3 use st_3        
def fillIn_StartBoundary(table, tablenum, dt, ht, ct, st_1):
    for col in range(0, len(table[tablenum][0])):
        nextI = max(0, I0 + col - dt)
        shortI = max(0, dt-I0-col)
        if(nextI > 0):
            cost = round(ct*col + ht*nextI + st_1*shortI + OptimalDec(table, 2, nextI),1)
            table[tablenum][0][col] = cost
        else:
            cost = round(ct*col + ht*nextI + st_1*shortI + OptimalDec(table, 2, 0),1)
            table[tablenum][0][col] = cost
    return table

#test instance 1 use st_1, test instance 2 use st_2, test instance 3 use st_3
def fillIn_EndBoundary(table, tablenum, dt, ht, ct, st_1):
    for row in range(0, len(table[tablenum])):
        for col in range(0, len(table[tablenum][row])):
            nextI = max(0, row + col - dt)
            shortI = max(0, dt-row-col)
            if(nextI > 0):
                cost = round(ct*col + ht*nextI, 1)
                table[tablenum][row][col] = cost
            else:
                cost = round(ct*col + st_1*shortI, 1)
                table[tablenum][row][col] = cost
    return table

#test instance 1 use st_1, test instance 2 use st_2, test instance 3 use st_3
def fillIn_Table(table, tablenum, dt, ht, ct, st_1): 
    for row in range(0, len(table[tablenum])):
        for col in range(0, len(table[tablenum][row])):
            nextI = max(0, row + col - dt)
            shortI = max(0, dt-row-col)
            if(nextI >= S):
                nextI = S
            if(shortI >= S):
                shortI = S
            nexttable = tablenum + 1
            if(nextI > 0):
                cost = round(ct*col + ht*nextI + OptimalDec(table, nexttable, nextI),1)
                table[tablenum][row][col] = cost
#                 print(f'nextI={nextI}, shortI={shortI}, nexttable={nexttable}, dec={OptimalDec(table, nexttable, nextI)}')
            else:
                cost = round(ct*col + st_1*shortI + OptimalDec(table, nexttable, 0),1)
                table[tablenum][row][col] = cost
#                 print(f'nextI={nextI}, shortI={shortI}, nexttable={nexttable}, dec={OptimalDec(table, nexttable, 0)}')
#                 print(f'SHORTING table={tablenum},ht={ht}, ct={ct}, st={st_1} dt={dt}, da={da}, row={row}, col={col}, nexttable={nexttable}, nextI={nextI}, shortI={shortI}, cost={cost}')
    return table

# Finds minimum VALUE in the row
def OptimalDec(table, tablenum, row):
    opt = 99999
    optindex = -1
    for col in range(0,len(table[tablenum][row])):
        if(table[tablenum][row][col] < opt):
            optindex = col
            opt = table[tablenum][row][col]
    return opt

# grabs the index of the minimum VALUE in the row
def findMinInRowIndex(table, tablenum, row):
    FinalCost = 99999
    indexOfFinalCost = -1
    for col in range(0,len(table[tablenum][row])):
        if(table[tablenum][row][col] < FinalCost):
            FinalCost = table[tablenum][row][col]
            indexOfFinalCost = col
#             print(f'indexOfFinalCost={col}')
    return indexOfFinalCost

def recursive(T, table, tablenum, dt, a, ht, ct, st_1):
    if(tablenum == 1):
#         print(f'tablenum={tablenum}, dt={dt}')
#         dts.append(dt)
        table = fillIn_StartBoundary(table, 1, dt[a], ht, ct, st_1)
        return table
    else:
#         print(f'tablenum={tablenum}, dt={dt}')
#         dts.append(dt)
        if(tablenum == T):
#             print(f'tablenum={tablenum}, dt={dt}')
            table = fillIn_EndBoundary(table, 10, dt[a], ht, ct, st_1)
#             dt = int(rd.uniform(0,20))
            tablenum = tablenum - 1
            a = a - 1
            return recursive(T, table, tablenum, dt, a, ht, ct, st_1)
        else:
#             da = int(rd.uniform(0,20))
            table = fillIn_Table(table, tablenum, dt[a], ht, ct, st_1)
#             dt = da
            tablenum = tablenum - 1
            a = a - 1
            return recursive(T, table, tablenum, dt, a, ht, ct, st_1)

#### Solve and Run DP

In [126]:
%%time
rd.seed(2844)

def createDummyTables(Case_1):
    for i in Case_1:
        dummy_fill_In(Case_1[i], i)
    return Case_1

def RunOnce(table, T, dtlist, a, ht, ct, st):
    Case_1 = recursive(T, table, T, dtlist, a, ht, ct, st)
    return Case_1

def findOptimalPath(table, var, Periods):
    n = 0
    li = []
    CurrInv = I0
    NextInv = 0 
    ShortInv = 0
    for t in range(1,(Periods+1)):
        if(t == 1):
            xVal = findMinInRowIndex(table, t, CurrInv)
            li.append([CurrInv, ShortInv, xVal, var[2][n]])
            NextInv = max(0, I0 + xVal - var[2][n])
            if(NextInv >= S):
                NextInv = S
            ShortInv = max(0, var[2][n]- I0 - xVal)
            if(ShortInv >= S):
                ShortInv = S
            if(NextInv > 0):
                NextInv = NextInv
            else:
                NextInv = 0
    #         print(f'table={t}, CurrInv={CurrInv}, NextInv={NextInv}, ShortI={ShortInv}, Col={xVal}, dts={dts[n]}')
        else:
            if(n == 0):
                xVal = findMinInRowIndex(table, t, CurrInv)
                li.append([CurrInv, ShortInv, xVal, var[2][n]])
                if(NextInv >= S):
                    NextInv = S
                ShortInv = max(0, var[2][n]- CurrInv - xVal)
                if(ShortInv >= S):
                    ShortInv = S
                if(NextInv > 0):
                    NextInv = NextInv
                else:
                    NextInv = 0
    #             print(f'table={t}, CurrInv={CurrInv}, NextInv={NextInv}, ShortI={ShortInv}, Col={xVal}, dts={dts[n]}')
            else:
                n = n-1
                CurrInv = NextInv
                if(CurrInv >= S):
                    CurrInv = S
                xVal = findMinInRowIndex(table, t, CurrInv)
                ShortInv = max(0, var[2][n]- CurrInv - xVal)
                li.append([CurrInv, ShortInv, xVal, var[2][n]])
                NextInv = max(0, CurrInv + xVal - var[2][n])
                if(NextInv >= S):
                    NextInv = S
                if(ShortInv >= S):
                    ShortInv = S
                if(NextInv > 0):
                    NextInv = NextInv
                else:
                    NextInv = 0
    #             print(f'table={t}, CurrInv={CurrInv}, NextInv={NextInv}, ShortI={ShortInv}, Col={xVal}, dts={dts[n]}')
    return li

def generateFrame(li, Periods, var, ind):
    # Generate Average + Standard Deviation over all the 10 instances, prodcution cost, holding cost, runtime
    totCost = []
    prodCost = []
    holdCost = []
    ShortCost = []
    tables = []
    AverageTotalCost = []

    CurrentInv= []
    ShortInv = []
    UnitsProd = []
    Demand = []
    for i in range(0, len(li)):
        CurrentInv.append(li[i][0])
        ShortInv.append(li[i][1])
        UnitsProd.append(li[i][2])
        Demand.append(li[i][3])

    for i in range(1, (Periods+1)): #test instance 1 use 11 and st_1, test instance 2 use 21 and st_2, test instance 3 use 31 and st_3
        tables.append(i)
        pc = round(var[0][ind]*li[i-1][2],1)
        prodCost.append(pc)
        hc = round(var[1][ind]*(max(0,(li[i-1][0]+li[i-1][2]-li[i-1][3]))),1)
        holdCost.append(hc)
        sc = round(var[3][ind]*(max(0,(li[i-1][3]-li[i-1][0]-li[i-1][2]))),1)
        ShortCost.append(sc)
        totCost.append(pc + hc + sc)
    df = pd.DataFrame({
            'Period':tables,
            'Current Inventory':CurrentInv,
            'Units Produced':UnitsProd,
            'Demand':Demand,
            'Unsatisfied Demand':ShortInv,
            'Production Cost' : prodCost,
            'Holding Cost' : holdCost,
            'Shortage Cost':ShortCost,
            'Total Cost In Period' : totCost
        })
    return df
#     SumCost = df.sum(axis=0)
#     AvgProdCost = df.mean(axis=0)
#     df

Wall time: 0 ns


In [127]:
var = generateVariables(T_1)
print(var[2])
Case_1 ={1:[],2:[],3:[],4:[],5:[],6:[],7:[],8:[],9:[],10:[]} 
Case_1 = createDummyTables(Case_1)
table = RunOnce(Case_1, T_1, var[2], 9, var[0][9], var[1][9], var[3][9])
# print(table)
l = findOptimalPath(table, var, T_1)
# print(l)
df = generateFrame(l, T_1, var, 9)
print(l)
SumCost = df.sum(axis=0)
AvgProdCost = df.mean(axis=0)
print(f'Average Production Cost = {round(AvgProdCost[5],1)} \nTotal Cost = {round(SumCost[8],1)}')
print(df)

[4, 4, 16, 18, 0, 7, 14, 7, 7, 7]
[[0, 0, 4, 4], [0, 0, 4, 4], [0, 0, 16, 4], [0, 0, 18, 4], [0, 0, 0, 4], [0, 4, 7, 4], [0, 0, 14, 4], [0, 0, 7, 4], [0, 0, 7, 4], [0, 0, 7, 4]]
Average Production Cost = 90.7 
Total Cost = 1405.6
   Period  Current Inventory  Units Produced  Demand  Unsatisfied Demand  \
0       1                  0               4       4                   0   
1       2                  0               4       4                   0   
2       3                  0              16       4                   0   
3       4                  0              18       4                   0   
4       5                  0               0       4                   0   
5       6                  0               7       4                   4   
6       7                  0              14       4                   0   
7       8                  0               7       4                   0   
8       9                  0               7       4                   0   
9      10 

### Test Cases

#### Instance 1, T=10

In [129]:
%%time
rd.seed(2844)

var = generateVariables(T_1)
dflist = []
Case_1 ={1:[],2:[],3:[],4:[],5:[],6:[],7:[],8:[],9:[],10:[]} 
for i in range(9,-1,-1): #(9,-1,-1):
    Case_1 = createDummyTables(Case_1)
    table = RunOnce(Case_1, T_1, var[2], 9, var[0][i], var[1][i], var[3][i])
    l = findOptimalPath(table, var, T_1)
    df = generateFrame(l, T_1, var, i)
    dflist.append(df)

AverageTotalCost = 0
AverageProdCost = 0
for i in range(0, T_1):    
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print('\n\n',dflist[i])
        SumCost = dflist[i].sum(axis=0)
        AvgProdCost = dflist[i].mean(axis=0)
        AverageTotalCost = AverageTotalCost + SumCost[8]
        AverageProdCost = AverageProdCost + AvgProdCost[5]
        print(f'Average Production Cost = {round(AvgProdCost[5],1)} \nTotal Cost = {round(SumCost[8],1)}')

print(f'\n\nAverage Total Cost = {round(AverageTotalCost/10,1)}\nAverage Production Cost = {round(AverageProdCost/10,1)}')



    Period  Current Inventory  Units Produced  Demand  Unsatisfied Demand  \
0       1                  0               4       4                   0   
1       2                  0               4       4                   0   
2       3                  0              16       4                   0   
3       4                  0              18       4                   0   
4       5                  0               0       4                   0   
5       6                  0               7       4                   4   
6       7                  0              14       4                   0   
7       8                  0               7       4                   0   
8       9                  0               7       4                   0   
9      10                  0               7       4                   0   

   Production Cost  Holding Cost  Shortage Cost  Total Cost In Period  
0             43.2           0.0              0                  43.2  
1             43

#### Instance 2, T=20

In [ ]:
%%time
rd.seed(2844)

var = generateVariables(T_2)
dflist = []
Case_1 ={1:[],2:[],3:[],4:[],5:[],6:[],7:[],8:[],9:[],10:[],
        11:[],12:[],13:[],14:[],15:[],16:[],17:[],18:[],19:[],20:[]} 
for i in range(19,9,-1):
    Case_1 = createDummyTables(Case_1)
    table = RunOnce(Case_1, T_2, var[2], 9, var[0][i], var[1][i], var[4][i])
    l = findOptimalPath(table, var, T_2)
    df = generateFrame(l, T_2, var, i)
    dflist.append(df)

AverageTotalCost = 0
AverageProdCost = 0
for i in range(0, T_2):    
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print('\n\n',dflist[i])
        SumCost = dflist[i].sum(axis=0)
        AvgProdCost = dflist[i].mean(axis=0)
        AverageTotalCost = AverageTotalCost + SumCost[8]
        AverageProdCost = AverageProdCost + AvgProdCost[5]
        print(f'Average Production Cost = {round(AvgProdCost[5],1)} \nTotal Cost = {round(SumCost[8],1)}')

print(f'\n\nAverage Total Cost = {round(AverageTotalCost/10,1)}\nAverage Production Cost = {round(AverageProdCost/10,1)}')

#### Instance 3, T=30

In [ ]:
%%time
rd.seed(2844)

var = generateVariables(T_3)
dflist = []
Case_1 ={1:[],2:[],3:[],4:[],5:[],6:[],7:[],8:[],9:[],10:[]} 
for i in range(29,19,-1):
    Case_1 = createDummyTables(Case_1)
    table = RunOnce(Case_1, T_3, var[2], 29, var[0][i], var[1][i], var[5][i])
    l = findOptimalPath(table, var, T_3)
    df = generateFrame(l, T_3, var, i)
    dflist.append(df)

AverageTotalCost = 0
AverageProdCost = 0
for i in range(0, T_3):    
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print('\n\n',dflist[i])
        SumCost = dflist[i].sum(axis=0)
        AvgProdCost = dflist[i].mean(axis=0)
        AverageTotalCost = AverageTotalCost + SumCost[8]
        AverageProdCost = AverageProdCost + AvgProdCost[5]
        print(f'Average Production Cost = {round(AvgProdCost[5],1)} \nTotal Cost = {round(SumCost[8],1)}')

print(f'\n\nAverage Total Cost = {round(AverageTotalCost/10,1)}\nAverage Production Cost = {round(AverageProdCost/10,1)}')